In [1]:
import os
import json
import re
import requests
import joblib
import numpy as np
import torch
from typing import TypedDict, List, Optional, Dict, Any, Tuple
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModel
from rank_bm25 import BM25Okapi

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

c:\Users\Playdata2\miniconda3\envs\final-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LLM Parser

In [14]:
import os, json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1) .env에서 OPENAI_API_KEY 불러오기
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2) 프롬프트 정의
parse_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 향수 쿼리 파서야.
사용자의 질문에서 다음 정보를 JSON 형식으로 추출해줘:
- brand: 브랜드명 (예: 샤넬, 디올, 입생로랑 등)
- concentration: (퍼퓸, 코롱 등)
- day_night_score: 사용시간 (주간, 야간, 데일리 등)
- gender: 성별 (남성, 여성, 유니섹스)
- season_score: 계절 (봄, 여름, 가을, 겨울)
- sizes: 용량 (30ml, 50ml, 100ml 등) 단위는 무시하고 숫자만

없는 값은 null로 두고, 반드시 유효한 JSON 형식으로만 응답해줘.

예시:
{{"brand": "샤넬", "gender": null, "sizes": "50ml", "season_score": null, "concentration": null, "day_night_score": null}}"""),
    ("user", "{query}")
])

def run_llm_parser(query: str):
    chain = parse_prompt | llm
    ai_response = chain.invoke({"query": query})
    response_text = ai_response.content.strip()

    # JSON 부분만 추출
    if "```json" in response_text:
        response_text = response_text.split("```json")[1].split("```")[0].strip()
    elif "```" in response_text:
        response_text = response_text.split("```")[1].strip()

    parsed = json.loads(response_text)
    return parsed

In [6]:

# 3) 테스트 실행
if __name__ == "__main__":
    query = "딥띠크 50ml 향수 있어?"
    parsed = run_llm_parser(query)
    print("원본 쿼리:", query)
    print("파싱 결과:", json.dumps(parsed, ensure_ascii=False, indent=2))


원본 쿼리: 딥띠크 50ml 향수 있어?
파싱 결과: {
  "brand": "딥띠크",
  "gender": null,
  "sizes": "50",
  "season_score": null,
  "concentration": null,
  "day_night_score": null
}


In [15]:
import re, json

# ---- 메타필터 함수들 ----
def filter_brand(brand_value):
    valid_brands = [
        '겔랑', '구찌', '끌로에', '나르시소 로드리게즈', '니샤네', '도르세', '디올', '딥티크', '랑콤',
        '로라 메르시에', '로에베', '록시땅', '르 라보', '메모', '메종 마르지엘라', '메종 프란시스 커정',
        '멜린앤게츠', '미우미우', '바이레도', '반클리프 아펠', '버버리', '베르사체', '불가리', '비디케이',
        '산타 마리아 노벨라', '샤넬', '세르주 루텐', '시슬리 코스메틱', '아쿠아 디 파르마', '에따 리브르 도량쥬',
        '에르메스', '에스티 로더', '엑스 니힐로', '이니시오 퍼퓸', '이솝', '입생로랑', '제르조프', '조 말론',
        '조르지오 아르마니', '줄리엣 헤즈 어 건', '지방시', '질 스튜어트', '크리드', '킬리안', '톰 포드',
        '티파니앤코', '퍼퓸 드 말리', '펜할리곤스', '프라다', '프레데릭 말'
    ]
    if brand_value is None:
        return None
    return brand_value if brand_value in valid_brands else None


def filter_concentration(concentration_value):
    valid_concentrations = ['솔리드 퍼퓸', '엑스트레 드 퍼퓸', '오 드 뚜왈렛', '오 드 코롱', '오 드 퍼퓸', '퍼퓸']
    if concentration_value is None:
        return None
    return concentration_value if concentration_value in valid_concentrations else None


def filter_day_night_score(day_night_value):
    valid_day_night = ["day", "night"]
    if day_night_value is None:
        return None
    if isinstance(day_night_value, str) and ',' in day_night_value:
        values = [v.strip() for v in day_night_value.split(',')]
        filtered_values = [v for v in values if v in valid_day_night]
        return ','.join(filtered_values) if filtered_values else None
    return day_night_value if day_night_value in valid_day_night else None


def filter_gender(gender_value):
    valid_genders = ['Female', 'Male', 'Unisex', 'unisex ']
    if gender_value is None:
        return None
    return gender_value if gender_value in valid_genders else None


def filter_season_score(season_value):
    valid_seasons = ['winter', 'spring', 'summer', 'fall']
    if season_value is None:
        return None
    return season_value if season_value in valid_seasons else None


def filter_sizes(sizes_value):
    """숫자만 추출해서 유효값인지 확인"""
    valid_sizes = ['30', '50', '75', '100', '150']
    if sizes_value is None:
        return None
    if isinstance(sizes_value, str):
        numbers = re.findall(r'\d+', sizes_value)
        for num in numbers:
            if num in valid_sizes:
                return num
    return str(sizes_value) if str(sizes_value) in valid_sizes else None


def apply_meta_filters(parsed_json: dict) -> dict:
    """파싱된 JSON에 메타필터링 적용"""
    if not parsed_json or "error" in parsed_json:
        return parsed_json
    
    return {
        'brand': filter_brand(parsed_json.get('brand')),
        'concentration': filter_concentration(parsed_json.get('concentration')),
        'day_night_score': filter_day_night_score(parsed_json.get('day_night_score')),
        'gender': filter_gender(parsed_json.get('gender')),
        'season_score': filter_season_score(parsed_json.get('season_score')),
        'sizes': filter_sizes(parsed_json.get('sizes'))
    }


In [16]:
from pinecone import Pinecone
import os

# Pinecone 초기화
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# perfume-vectordb2 인덱스 지정
index = pc.Index("perfume-vectordb2")


def build_pinecone_filter(filtered_json: dict) -> dict:
    """
    메타필터링 결과를 Pinecone filter dict로 변환
    """
    pinecone_filter = {}
    if filtered_json.get("brand"):
        pinecone_filter["brand"] = {"$eq": filtered_json["brand"]}
    if filtered_json.get("sizes"):
        pinecone_filter["sizes"] = {"$eq": filtered_json["sizes"]}
    if filtered_json.get("season_score"):
        pinecone_filter["season_score"] = {"$eq": filtered_json["season_score"]}
    if filtered_json.get("gender"):
        pinecone_filter["gender"] = {"$eq": filtered_json["gender"]}
    if filtered_json.get("concentration"):
        pinecone_filter["concentration"] = {"$eq": filtered_json["concentration"]}
    if filtered_json.get("day_night_score"):
        pinecone_filter["day_night_score"] = {"$eq": filtered_json["day_night_score"]}
    return pinecone_filter


def query_pinecone(vector, filtered_json: dict, top_k: int = 5):
    """
    Pinecone 벡터 검색 + 메타데이터 필터 적용
    """
    pinecone_filter = build_pinecone_filter(filtered_json)

    result = index.query(
        vector=vector,
        top_k=top_k,
        include_metadata=True,
        filter=pinecone_filter if pinecone_filter else None
    )
    return result


In [ ]:
# 여기서 부터 혼돈시작

In [1]:
# pip install -U pinecone-client openai python-dotenv
import os, json
from typing import Dict, Any, List
from dotenv import load_dotenv
from pinecone import Pinecone, PineconeApiException
from openai import OpenAI

# ========= 0) ENV & CLIENTS =========
load_dotenv()
oa = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("perfume-vectordb2")
EMBED_MODEL = "text-embedding-3-small"

# ========= 1) FILTER BUILDER =========
def build_pinecone_filter(filtered_json: Dict[str, Any]) -> Dict[str, Any]:
    """
    Pinecone 메타필터.
    - sizes: 배열/스칼라 모두 커버하기 위해 $in 사용 (["50"] 형태)
    - 나머지는 $eq
    """
    f: Dict[str, Any] = {}
    if filtered_json.get("brand"):
        f["brand"] = {"$eq": filtered_json["brand"]}
    if filtered_json.get("sizes"):
        f["sizes"] = {"$in": [str(filtered_json["sizes"])]}  # 배열/스칼라 모두 안전
    if filtered_json.get("season_score"):
        f["season_score"] = {"$eq": filtered_json["season_score"]}
    if filtered_json.get("gender"):
        f["gender"] = {"$eq": filtered_json["gender"]}
    if filtered_json.get("concentration"):
        f["concentration"] = {"$eq": filtered_json["concentration"]}
    if filtered_json.get("day_night_score"):
        f["day_night_score"] = {"$eq": filtered_json["day_night_score"]}
    return f

# ========= 2) QUERY =========
def query_pinecone(vector: List[float], filtered_json: Dict[str, Any], top_k: int = 5):
    """
    Pinecone 쿼리 (메타필터 + 문서 메타데이터 포함).
    필터 오류 시 sizes 필터 제거 후 재시도하고, 결과는 클라이언트에서 후필터링.
    """
    primary_filter = build_pinecone_filter(filtered_json) or None
    try:
        return index.query(
            vector=vector,
            top_k=top_k,
            include_metadata=True,
            filter=primary_filter
        )
    except PineconeApiException as e:
        # 필터 연산/타입 이슈 등으로 400이 날 수 있으므로 sizes만 제거하고 재시도
        if primary_filter and "sizes" in primary_filter:
            fallback_filter = dict(primary_filter)
            fallback_filter.pop("sizes", None)
            res = index.query(
                vector=vector,
                top_k=top_k,
                include_metadata=True,
                filter=fallback_filter if fallback_filter else None
            )
            # 클라이언트 후필터: sizes 일치만 남김
            want_size = str(filtered_json.get("sizes"))
            if want_size:
                res["matches"] = [
                    m for m in res.get("matches", [])
                    if want_size in (m.get("metadata", {}).get("sizes") or [])
                    or str(m.get("metadata", {}).get("sizes")) == want_size
                ]
            return res
        else:
            raise

# ========= 3) PRETTY PRINT =========
def print_docs_only(res):
    print("=== 검색된 문서들 ===")
    for i, m in enumerate(res.get("matches", []), start=1):
        meta = m.get("metadata", {}) or {}
        doc = {"id": m.get("id")}
        doc.update(meta)
        print(f"\n[{i}]")
        print(json.dumps(doc, ensure_ascii=False, indent=2))

# ========= 4) DEMO =========
if __name__ == "__main__":
    filtered = {
        "brand": "딥티크",
        "concentration": None,
        "day_night_score": None,
        "gender": None,
        "season_score": None,
        "sizes": "50"
    }
    query_text = "딥티크 50ml 향수 있어?"

    embed = oa.embeddings.create(model=EMBED_MODEL, input=query_text)
    vector = embed.data[0].embedding

    res = query_pinecone(vector, filtered, top_k=5)
    print_docs_only(res)


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== 검색된 문서들 ===

[1]
{
  "id": "perfume_3009ee702f886c68",
  "brand": "딥티크",
  "concentration": "오 드 뚜왈렛",
  "day_night_score": "night",
  "gender": "Unisex",
  "name": "오 데 썽 오 드 뚜왈렛",
  "no": 143.0,
  "season_score": "winter",
  "sizes": [
    "50",
    "100"
  ],
  "text": "감각의 물 또는 에센스의 물로 해석되는 향"
}

[2]
{
  "id": "perfume_8c25220938bc55f8",
  "brand": "딥티크",
  "concentration": "오 드 뚜왈렛",
  "day_night_score": "night",
  "gender": "Unisex",
  "name": "필로시코스 오 드 뚜왈렛",
  "no": 161.0,
  "season_score": "spring",
  "sizes": [
    "50",
    "100"
  ],
  "text": "신선하고 달콤한 무화과의 모든것을 느낄 수 있는 향"
}

[3]
{
  "id": "perfume_3e3c134ca8e46fac",
  "brand": "딥티크",
  "concentration": "오 드 뚜왈렛",
  "day_night_score": "day",
  "gender": "Unisex",
  "name": "롬브르 단 로 오 드 뚜왈렛",
  "no": 138.0,
  "season_score": "spring",
  "sizes": [
    "50",
    "100"
  ],
  "text": "비 오는 날 뿌리기 좋은 쌉싸름한 장미잎향"
}

[4]
{
  "id": "perfume_d52dc5f679a0ff96",
  "brand": "딥티크",
  "concentration": "오 드 뚜왈렛",
  "day_night_score": 

# LLM parser

In [23]:
import os, json, re
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from pinecone import Pinecone

# 환경 변수 로드
load_dotenv()

# LLM 및 임베딩 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Pinecone 초기화
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("perfume-vectordb2")

# ========== LLM 파서 ==========
parse_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 향수 쿼리 파서야.
사용자의 질문에서 다음 정보를 JSON 형식으로 추출해줘:
- brand: 브랜드명 (예: 샤넬, 디올, 입생로랑 등)
- concentration: (퍼퓸, 코롱 등)
- day_night_score: 사용시간 (주간, 야간, 데일리 등)
- gender: 성별 (남성, 여성, 유니섹스)
- season_score: 계절 (봄, 여름, 가을, 겨울)
- sizes: 용량 (30ml, 50ml, 100ml 등) 단위는 무시하고 숫자만

없는 값은 null로 두고, 반드시 유효한 JSON 형식으로만 응답해줘.

예시:
{{"brand": "샤넬", "gender": null, "sizes": "50", "season_score": null, "concentration": null, "day_night_score": null}}"""),
    ("user", "{query}")
])

def run_llm_parser(query: str):
    """사용자 쿼리를 JSON으로 파싱"""
    try:
        chain = parse_prompt | llm
        ai_response = chain.invoke({"query": query})
        response_text = ai_response.content.strip()

        # JSON 부분만 추출
        if "```json" in response_text:
            response_text = response_text.split("```json")[1].split("```")[0].strip()
        elif "```" in response_text:
            response_text = response_text.split("```")[1].strip()

        parsed = json.loads(response_text)
        return parsed
    except Exception as e:
        return {"error": f"파싱 오류: {str(e)}"}

# ========== 메타필터 함수들 ==========
def filter_brand(brand_value):
    valid_brands = [
        '겔랑', '구찌', '끌로에', '나르시소 로드리게즈', '니샤네', '도르세', '디올', '딥티크', '랑콤',
        '로라 메르시에', '로에베', '록시땅', '르 라보', '메모', '메종 마르지엘라', '메종 프란시스 커정',
        '멜린앤게츠', '미우미우', '바이레도', '반클리프 아펠', '버버리', '베르사체', '불가리', '비디케이',
        '산타 마리아 노벨라', '샤넬', '세르주 루텐', '시슬리 코스메틱', '아쿠아 디 파르마', '에따 리브르 도량쥬',
        '에르메스', '에스티 로더', '엑스 니힐로', '이니시오 퍼퓸', '이솝', '입생로랑', '제르조프', '조 말론',
        '조르지오 아르마니', '줄리엣 헤즈 어 건', '지방시', '질 스튜어트', '크리드', '킬리안', '톰 포드',
        '티파니앤코', '퍼퓸 드 말리', '펜할리곤스', '프라다', '프레데릭 말'
    ]
    if brand_value is None:
        return None
    return brand_value if brand_value in valid_brands else None

def filter_concentration(concentration_value):
    valid_concentrations = ['솔리드 퍼퓸', '엑스트레 드 퍼퓸', '오 드 뚜왈렛', '오 드 코롱', '오 드 퍼퓸', '퍼퓸']
    if concentration_value is None:
        return None
    return concentration_value if concentration_value in valid_concentrations else None

def filter_day_night_score(day_night_value):
    valid_day_night = ["day", "night"]
    if day_night_value is None:
        return None
    if isinstance(day_night_value, str) and ',' in day_night_value:
        values = [v.strip() for v in day_night_value.split(',')]
        filtered_values = [v for v in values if v in valid_day_night]
        return ','.join(filtered_values) if filtered_values else None
    return day_night_value if day_night_value in valid_day_night else None

def filter_gender(gender_value):
    valid_genders = ['Female', 'Male', 'Unisex', 'unisex ']
    if gender_value is None:
        return None
    return gender_value if gender_value in valid_genders else None

def filter_season_score(season_value):
    valid_seasons = ['winter', 'spring', 'summer', 'fall']
    if season_value is None:
        return None
    return season_value if season_value in valid_seasons else None

def filter_sizes(sizes_value):
    """숫자만 추출해서 유효값인지 확인"""
    valid_sizes = ['30', '50', '75', '100', '150']
    if sizes_value is None:
        return None
    if isinstance(sizes_value, str):
        numbers = re.findall(r'\d+', sizes_value)
        for num in numbers:
            if num in valid_sizes:
                return num
    return str(sizes_value) if str(sizes_value) in valid_sizes else None

def apply_meta_filters(parsed_json: dict) -> dict:
    """파싱된 JSON에 메타필터링 적용"""
    if not parsed_json or "error" in parsed_json:
        return parsed_json
    
    return {
        'brand': filter_brand(parsed_json.get('brand')),
        'concentration': filter_concentration(parsed_json.get('concentration')),
        'day_night_score': filter_day_night_score(parsed_json.get('day_night_score')),
        'gender': filter_gender(parsed_json.get('gender')),
        'season_score': filter_season_score(parsed_json.get('season_score')),
        'sizes': filter_sizes(parsed_json.get('sizes'))
    }

# ========== Pinecone 검색 함수들 ==========
def build_pinecone_filter(filtered_json: dict) -> dict:
    """메타필터링 결과를 Pinecone filter dict로 변환"""
    pinecone_filter = {}
    if filtered_json.get("brand"):
        pinecone_filter["brand"] = {"$eq": filtered_json["brand"]}
    if filtered_json.get("sizes"):
        pinecone_filter["sizes"] = {"$eq": filtered_json["sizes"]}
    if filtered_json.get("season_score"):
        pinecone_filter["season_score"] = {"$eq": filtered_json["season_score"]}
    if filtered_json.get("gender"):
        pinecone_filter["gender"] = {"$eq": filtered_json["gender"]}
    if filtered_json.get("concentration"):
        pinecone_filter["concentration"] = {"$eq": filtered_json["concentration"]}
    if filtered_json.get("day_night_score"):
        pinecone_filter["day_night_score"] = {"$eq": filtered_json["day_night_score"]}
    return pinecone_filter

def query_pinecone(vector, filtered_json: dict, top_k: int = 5):
    """Pinecone 벡터 검색 + 메타데이터 필터 적용"""
    pinecone_filter = build_pinecone_filter(filtered_json)
    
    result = index.query(
        vector=vector,
        top_k=top_k,
        include_metadata=True,
        filter=pinecone_filter if pinecone_filter else None
    )
    return result

# ========== RAG 응답 생성 ==========
response_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 향수 전문가야. 사용자의 질문에 대해 검색된 향수 정보를 바탕으로 친절하고 전문적인 추천을 해줘.

추천할 때 다음을 포함해줘:
1. 왜 이 향수를 추천하는지
2. 향의 특징과 느낌
3. 어떤 상황에 적합한지
4. 가격대나 용량 관련 조언 (있다면)

자연스럽고 친근한 톤으로 답변해줘."""),
    ("user", """사용자 질문: {original_query}

검색된 향수 정보:
{search_results}

위 정보를 바탕으로 향수를 추천해줘.""")
])

def format_search_results(pinecone_results):
    """Pinecone 검색 결과를 텍스트로 포맷팅"""
    if not pinecone_results or not pinecone_results.get('matches'):
        return "검색된 향수가 없습니다."
    
    formatted_results = []
    for i, match in enumerate(pinecone_results['matches'], 1):
        metadata = match.get('metadata', {})
        score = match.get('score', 0)
        
        result_text = f"""
{i}. 향수명: {metadata.get('perfume_name', '정보없음')}
   - 브랜드: {metadata.get('brand', '정보없음')}
   - 성별: {metadata.get('gender', '정보없음')}
   - 용량: {metadata.get('sizes', '정보없음')}ml
   - 계절: {metadata.get('season_score', '정보없음')}
   - 사용시간: {metadata.get('day_night_score', '정보없음')}
   - 농도: {metadata.get('concentration', '정보없음')}
   - 유사도 점수: {score:.3f}
"""
        formatted_results.append(result_text.strip())
    
    return "\n\n".join(formatted_results)

def generate_response(original_query: str, search_results):
    """검색 결과를 바탕으로 최종 응답 생성"""
    try:
        formatted_results = format_search_results(search_results)
        
        chain = response_prompt | llm
        response = chain.invoke({
            "original_query": original_query,
            "search_results": formatted_results
        })
        
        return response.content
    except Exception as e:
        return f"응답 생성 중 오류가 발생했습니다: {str(e)}"

# ========== 통합 RAG 파이프라인 ==========
def perfume_rag_pipeline(user_query: str, top_k: int = 5):
    """
    완전한 향수 추천 RAG 파이프라인
    
    Args:
        user_query (str): 사용자 질문
        top_k (int): 반환할 향수 개수
    
    Returns:
        dict: 단계별 결과와 최종 추천
    """
    print(f"🔍 사용자 쿼리: {user_query}")
    
    # 1단계: LLM으로 쿼리 파싱
    print("\n1️⃣ 쿼리 파싱 중...")
    parsed_json = run_llm_parser(user_query)
    print(f"파싱 결과: {json.dumps(parsed_json, ensure_ascii=False, indent=2)}")
    
    if "error" in parsed_json:
        return {
            "error": "쿼리 파싱에 실패했습니다.",
            "details": parsed_json
        }
    
    # 2단계: 메타필터 적용
    print("\n2️⃣ 메타필터 적용 중...")
    filtered_json = apply_meta_filters(parsed_json)
    print(f"필터링 결과: {json.dumps(filtered_json, ensure_ascii=False, indent=2)}")
    
    # 3단계: 쿼리 벡터화
    print("\n3️⃣ 쿼리 벡터화 중...")
    try:
        query_vector = embeddings.embed_query(user_query)
        print(f"벡터 차원: {len(query_vector)}")
    except Exception as e:
        return {
            "error": "쿼리 벡터화에 실패했습니다.",
            "details": str(e)
        }
    
    # 4단계: Pinecone 검색
    print("\n4️⃣ 벡터 검색 중...")
    try:
        search_results = query_pinecone(query_vector, filtered_json, top_k)
        print(f"검색된 향수 개수: {len(search_results.get('matches', []))}")
    except Exception as e:
        return {
            "error": "벡터 검색에 실패했습니다.",
            "details": str(e)
        }
    
    # 5단계: 최종 응답 생성
    print("\n5️⃣ 최종 응답 생성 중...")
    final_response = generate_response(user_query, search_results)
    
    return {
        "original_query": user_query,
        "parsed_query": parsed_json,
        "filtered_query": filtered_json,
        "search_results": search_results,
        "recommendation": final_response,
        "status": "success"
    }

# ========== 사용 예제 ==========
if __name__ == "__main__":
    # 테스트 쿼리들
    test_queries = [
        "샤넬 50ml 여성용 향수 추천해줘",
        "겨울에 사용하기 좋은 남성 향수가 뭐가 있을까?",
        "디올에서 나온 퍼퓸 중에 30ml짜리로 추천해줘",
        "밤에 사용하기 좋은 유니섹스 향수 찾고 있어"
    ]
    
    for query in test_queries:
        print("="*80)
        result = perfume_rag_pipeline(query)
        
        if result.get("status") == "success":
            print(f"\n🎯 최종 추천:\n{result['recommendation']}")
        else:
            print(f"\n❌ 오류: {result.get('error')}")
        
        print("\n" + "="*80 + "\n")

# ========== 간단한 실행 함수 ==========
def ask_perfume(query: str):
    """간단한 향수 검색 함수 - DB 결과만 반환"""
    result = perfume_rag_pipeline(query)
    if result.get("status") == "success":
        return result["formatted_results"]
    else:
        return f"죄송합니다. {result.get('error', '알 수 없는 오류가 발생했습니다.')}"

🔍 사용자 쿼리: 샤넬 50ml 여성용 향수 추천해줘

1️⃣ 쿼리 파싱 중...
파싱 결과: {
  "brand": "샤넬",
  "gender": "여성",
  "sizes": "50",
  "season_score": null,
  "concentration": null,
  "day_night_score": null
}

2️⃣ 메타필터 적용 중...
필터링 결과: {
  "brand": "샤넬",
  "concentration": null,
  "day_night_score": null,
  "gender": null,
  "season_score": null,
  "sizes": "50"
}

3️⃣ 쿼리 벡터화 중...
벡터 차원: 1536

4️⃣ 벡터 검색 중...
검색된 향수 개수: 5

5️⃣ 최종 응답 생성 중...

🎯 최종 추천:
안녕하세요! 샤넬의 50ml 여성용 향수를 찾고 계시군요. 여러 옵션이 있지만, 제가 추천드리고 싶은 향수는 **샤넬의 오 드 퍼퓸**입니다. 

### 추천 이유
샤넬은 고급스러움과 우아함을 상징하는 브랜드로, 그들의 향수는 항상 뛰어난 품질과 독창성을 자랑합니다. 특히 오 드 퍼퓸은 농도가 높아 향이 오래 지속되며, 깊이 있는 향을 즐길 수 있습니다.

### 향의 특징과 느낌
이 향수는 가을에 잘 어울리는 따뜻하고 풍부한 향을 가지고 있습니다. 일반적으로 샤넬의 오 드 퍼퓸은 플로럴과 우디 노트가 조화를 이루며, 부드럽고 세련된 느낌을 줍니다. 특히, 여성스러움을 강조하면서도 강렬한 인상을 남기는 매력이 있습니다.

### 적합한 상황
이 향수는 주로 낮에 사용하기 적합합니다. 일상적인 외출이나 직장, 혹은 특별한 모임에서도 잘 어울립니다. 가을의 쌀쌀한 날씨와 함께하면 더욱 매력적인 향을 발산할 수 있습니다.

### 가격대 및 용량 조언
50ml 용량은 일반적으로 사용하기에 적당한 사이즈로, 가격대는 보통 10만원대 중반에서 후반에 형성되어 있습니다. 샤넬의 향수는 품질이 뛰어나기 때문에, 이 가격대

In [ ]:
# pip install -U langchain langgraph langchain-openai tiktoken
import os, json
from typing import TypedDict, List, Optional, Dict, Any

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# ---------- 0) Config ----------
os.environ.setdefault("OPENAI_API_KEY", "PUT_YOUR_KEY_HERE")  # or set in env
MODEL_NAME = "gpt-4o-mini"  # keep it small & fast for routing

SUPERVISOR_SYSTEM_PROMPT = """
You are the “Perfume Recommendation Supervisor (Router)”. Analyze the user’s query (Korean or English) and route to exactly ONE agent below.

[Agents]
- LLM_parser         : Parses/normalizes multi-facet queries (2+ product facets).
- FAQ_agent          : Perfume knowledge / definitions / differences / general questions.
- human_fallback     : Non-perfume or off-topic queries.
- price_agent        : Price-only intents (cheapest, price, buy, discount, etc.).
- ML_agent           : Single-preference recommendations (mood/season vibe like “fresh summer”, “sweet”, etc.).

[Facets to detect (“product facets”)]
- brand            (e.g., Chanel, Dior, Creed)
- season           (spring/summer/fall/winter; “for summer/winter”)
- gender           (male/female/unisex)
- sizes            (volume in ml: 30/50/100 ml)
- day_night_score  (day/night/daily/office/club, etc.)
- concentration    (EDT/EDP/Extrait/Parfum/Cologne)

[Price intent keywords (not exhaustive)]
- Korean: 가격, 최저가, 얼마, 가격대, 구매, 판매, 할인, 어디서 사, 배송비
- English: price, cost, cheapest, buy, purchase, discount

[FAQ examples]
- Differences between EDP vs EDT, note definitions, longevity/projection, brand/line info.

[Single-preference (ML_agent) examples]
- “Recommend a cool perfume for summer”, “Recommend a sweet scent”, “One citrusy fresh pick”
  (= 0–1 of the above facets mentioned; primarily taste/mood/situation).

[Routing rules (priority)]
1) Non-perfume / off-topic → human_fallback
2) Clear price-only intent (even if one facet is present as context) → price_agent
   e.g., “Chanel No. 5 50ml cheapest price?” → price_agent
3) Count product facets in the query:
   - If facets ≥ 2 → LLM_parser
4) Otherwise (single-topic queries):
   - Perfume knowledge/definitions → FAQ_agent
   - Single taste/mood recommendation → ML_agent
5) Tie-breakers:
   - If price intent is clear → price_agent
   - If facets ≥ 2 → LLM_parser
   - Else: knowledge → FAQ_agent, taste → ML_agent

[Output format]
Return ONLY this JSON (no extra text):
{{
  "next": "<LLM_parser|FAQ_agent|human_fallback|price_agent|ML_agent>",
  "reason": "<one short English sentence>",
  "facet_count": <integer>,
  "facets": {{
    "brand": "<value or null>",
    "season": "<value or null>",
    "gender": "<value or null>",
    "sizes": "<value or null>",
    "day_night_score": "<value or null>",
    "concentration": "<value or null>"
  }},
  "scent_vibe": "<value if detected, else null>",
  "query_intent": "<price|faq|scent_pref|non_perfume|other>"
}}
""".strip()

# ---------- 1) State ----------
class AgentState(TypedDict):
    messages: List[BaseMessage]           # conversation log
    next: Optional[str]                   # routing decision key
    router_json: Optional[Dict[str, Any]] # parsed JSON from router

# ---------- 2) LLM (Supervisor) ----------
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SUPERVISOR_SYSTEM_PROMPT),
        ("user", "{query}")
    ]
)

def supervisor_node(state: AgentState) -> AgentState:
    """Call the router LLM and return parsed JSON + routing target."""
    user_query = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            user_query = m.content
            break
    if not user_query:
        user_query = "(empty)"

    chain = router_prompt | llm
    ai = chain.invoke({"query": user_query})
    text = ai.content

    # JSON strict parse
    chosen = "human_fallback"
    parsed: Dict[str, Any] = {}
    try:
        parsed = json.loads(text)
        maybe = parsed.get("next")
        if isinstance(maybe, str) and maybe in {"LLM_parser","FAQ_agent","human_fallback","price_agent","ML_agent"}:
            chosen = maybe
    except Exception:
        parsed = {"error": "invalid_json", "raw": text}

    msgs = state["messages"] + [AIMessage(content=text)]
    return {
        "messages": msgs,
        "next": chosen,
        "router_json": parsed
    }

# ---------- 3) Mock Agent Nodes (for testing) ----------
def passthrough(name: str):
    def _node(state: AgentState) -> AgentState:
        payload = state.get("router_json") or {}
        summary = f"[{name}] handled. reason={payload.get('reason')} facets={payload.get('facets')} intent={payload.get('query_intent')}"
        msgs = state["messages"] + [AIMessage(content=summary)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
    return _node

LLM_parser      = passthrough("LLM_parser")
FAQ_agent       = passthrough("FAQ_agent")
human_fallback  = passthrough("human_fallback")
price_agent     = passthrough("price_agent")
ML_agent        = passthrough("ML_agent")

# ---------- 4) Build Graph ----------
graph = StateGraph(AgentState)

graph.add_node("supervisor", supervisor_node)
graph.add_node("LLM_parser", LLM_parser)
graph.add_node("FAQ_agent", FAQ_agent)
graph.add_node("human_fallback", human_fallback)
graph.add_node("price_agent", price_agent)
graph.add_node("ML_agent", ML_agent)

graph.set_entry_point("supervisor")

# Conditional routing
def router_edge(state: AgentState) -> str:
    return state["next"] or "human_fallback"

graph.add_conditional_edges(
    "supervisor",
    router_edge,
    {
        "LLM_parser": "LLM_parser",
        "FAQ_agent": "FAQ_agent",
        "human_fallback": "human_fallback",
        "price_agent": "price_agent",
        "ML_agent": "ML_agent",
    },
)

# End states
for node in ["LLM_parser", "FAQ_agent", "human_fallback", "price_agent", "ML_agent"]:
    graph.add_edge(node, END)

app = graph.compile()

# ---------- 5) Batch Test ----------
TEST_QUERIES = [
    "입생로랑 여성용 50ml 겨울용 향수 추천해줘.",                 
    "디올 EDP로 가을 밤(야간)에 쓸 만한 향수 있어?",                
    "EDP랑 EDT 차이가 뭐야?",                                       
    "탑노트·미들노트·베이스노트가 각각 무슨 뜻이야?",               
    "오늘 점심 뭐 먹을까?",                                         
    "오늘 서울 날씨 어때?",                                         
    "샤넬 넘버5 50ml 최저가 알려줘.",                               
    "디올 소바쥬 가격 얼마야? 어디서 사는 게 제일 싸?",             
    "여름에 시원한 향수 추천해줘.",                                 
    "달달한 향 추천해줘.",                                         
]

def run_tests():
    for q in TEST_QUERIES:
        print("="*80)
        print("Query:", q)
        init: AgentState = {
            "messages": [HumanMessage(content=q)],
            "next": None,
            "router_json": None
        }
        out = app.invoke(init)
        ai_msgs = [m for m in out["messages"] if isinstance(m, AIMessage)]
        router_raw = ai_msgs[-2].content if len(ai_msgs) >= 2 else "(no router output)"
        agent_summary = ai_msgs[-1].content if ai_msgs else "(no agent output)"
        print("Router JSON:", router_raw)
        print("Agent summary:", agent_summary)

if __name__ == "__main__":
    run_tests()


In [2]:
# 연결버전
# pip install -U langchain langgraph langchain-openai tiktoken python-dotenv pinecone-client
import os, json, re
from typing import TypedDict, List, Optional, Dict, Any
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from pinecone import Pinecone

# 환경 변수 로드
load_dotenv()

# ---------- 0) Config ----------
os.environ.setdefault("OPENAI_API_KEY", "PUT_YOUR_KEY_HERE")  # or set in env
MODEL_NAME = "gpt-4o-mini"  # keep it small & fast for routing

SUPERVISOR_SYSTEM_PROMPT = """
You are the "Perfume Recommendation Supervisor (Router)". Analyze the user's query (Korean or English) and route to exactly ONE agent below.

[Agents]
- LLM_parser         : Parses/normalizes multi-facet queries (2+ product facets).
- FAQ_agent          : Perfume knowledge / definitions / differences / general questions.
- human_fallback     : Non-perfume or off-topic queries.
- price_agent        : Price-only intents (cheapest, price, buy, discount, etc.).
- ML_agent           : Single-preference recommendations (mood/season vibe like "fresh summer", "sweet", etc.).

[Facets to detect ("product facets")]
- brand            (e.g., Chanel, Dior, Creed)
- season           (spring/summer/fall/winter; "for summer/winter")
- gender           (male/female/unisex)
- sizes            (volume in ml: 30/50/100 ml)
- day_night_score  (day/night/daily/office/club, etc.)
- concentration    (EDT/EDP/Extrait/Parfum/Cologne)

[Price intent keywords (not exhaustive)]
- Korean: 가격, 최저가, 얼마, 가격대, 구매, 판매, 할인, 어디서 사, 배송비
- English: price, cost, cheapest, buy, purchase, discount

[FAQ examples]
- Differences between EDP vs EDT, note definitions, longevity/projection, brand/line info.

[Single-preference (ML_agent) examples]
- "Recommend a cool perfume for summer", "Recommend a sweet scent", "One citrusy fresh pick"
  (= 0–1 of the above facets mentioned; primarily taste/mood/situation).

[Routing rules (priority)]
1) Non-perfume / off-topic → human_fallback
2) Clear price-only intent (even if one facet is present as context) → price_agent
   e.g., "Chanel No. 5 50ml cheapest price?" → price_agent
3) Count product facets in the query:
   - If facets ≥ 2 → LLM_parser
4) Otherwise (single-topic queries):
   - Perfume knowledge/definitions → FAQ_agent
   - Single taste/mood recommendation → ML_agent
5) Tie-breakers:
   - If price intent is clear → price_agent
   - If facets ≥ 2 → LLM_parser
   - Else: knowledge → FAQ_agent, taste → ML_agent

[Output format]
Return ONLY this JSON (no extra text):
{{
  "next": "<LLM_parser|FAQ_agent|human_fallback|price_agent|ML_agent>",
  "reason": "<one short English sentence>",
  "facet_count": <integer>,
  "facets": {{
    "brand": "<value or null>",
    "season": "<value or null>",
    "gender": "<value or null>",
    "sizes": "<value or null>",
    "day_night_score": "<value or null>",
    "concentration": "<value or null>"
  }},
  "scent_vibe": "<value if detected, else null>",
  "query_intent": "<price|faq|scent_pref|non_perfume|other>"
}}
""".strip()

# ---------- 1) State ----------
class AgentState(TypedDict):
    messages: List[BaseMessage]           # conversation log
    next: Optional[str]                   # routing decision key
    router_json: Optional[Dict[str, Any]] # parsed JSON from router

# ---------- 2) LLM 초기화 ----------
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Pinecone 초기화
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("perfume-vectordb2")

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SUPERVISOR_SYSTEM_PROMPT),
        ("user", "{query}")
    ]
)

def supervisor_node(state: AgentState) -> AgentState:
    """Call the router LLM and return parsed JSON + routing target."""
    user_query = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            user_query = m.content
            break
    if not user_query:
        user_query = "(empty)"

    chain = router_prompt | llm
    ai = chain.invoke({"query": user_query})
    text = ai.content

    # JSON strict parse
    chosen = "human_fallback"
    parsed: Dict[str, Any] = {}
    try:
        parsed = json.loads(text)
        maybe = parsed.get("next")
        if isinstance(maybe, str) and maybe in {"LLM_parser","FAQ_agent","human_fallback","price_agent","ML_agent"}:
            chosen = maybe
    except Exception:
        parsed = {"error": "invalid_json", "raw": text}

    msgs = state["messages"] + [AIMessage(content=text)]
    return {
        "messages": msgs,
        "next": chosen,
        "router_json": parsed
    }

# ---------- 3) RAG Pipeline Functions ----------
parse_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 향수 쿼리 파서야.
사용자의 질문에서 다음 정보를 JSON 형식으로 추출해줘:
- brand: 브랜드명 (예: 샤넬, 디올, 입생로랑 등)
- concentration: (퍼퓸, 코롱 등)
- day_night_score: 사용시간 (주간, 야간, 데일리 등)
- gender: 성별 (남성, 여성, 유니섹스)
- season_score: 계절 (봄, 여름, 가을, 겨울)
- sizes: 용량 (30ml, 50ml, 100ml 등) 단위는 무시하고 숫자만

없는 값은 null로 두고, 반드시 유효한 JSON 형식으로만 응답해줘.

예시:
{{"brand": "샤넬", "gender": null, "sizes": "50", "season_score": null, "concentration": null, "day_night_score": null}}"""),
    ("user", "{query}")
])

def run_llm_parser(query: str):
    """사용자 쿼리를 JSON으로 파싱"""
    try:
        chain = parse_prompt | llm
        ai_response = chain.invoke({"query": query})
        response_text = ai_response.content.strip()

        # JSON 부분만 추출
        if "```json" in response_text:
            response_text = response_text.split("```json")[1].split("```")[0].strip()
        elif "```" in response_text:
            response_text = response_text.split("```")[1].strip()

        parsed = json.loads(response_text)
        return parsed
    except Exception as e:
        return {"error": f"파싱 오류: {str(e)}"}

# 메타필터 함수들
def filter_brand(brand_value):
    valid_brands = [
        '겔랑', '구찌', '끌로에', '나르시소 로드리게즈', '니샤네', '도르세', '디올', '딥티크', '랑콤',
        '로라 메르시에', '로에베', '록시땅', '르 라보', '메모', '메종 마르지엘라', '메종 프란시스 커정',
        '멜린앤게츠', '미우미우', '바이레도', '반클리프 아펠', '버버리', '베르사체', '불가리', '비디케이',
        '산타 마리아 노벨라', '샤넬', '세르주 루텐', '시슬리 코스메틱', '아쿠아 디 파르마', '에따 리브르 도량쥬',
        '에르메스', '에스티 로더', '엑스 니힐로', '이니시오 퍼퓸', '이솝', '입생로랑', '제르조프', '조 말론',
        '조르지오 아르마니', '줄리엣 헤즈 어 건', '지방시', '질 스튜어트', '크리드', '킬리안', '톰 포드',
        '티파니앤코', '퍼퓸 드 말리', '펜할리곤스', '프라다', '프레데릭 말'
    ]
    if brand_value is None:
        return None
    return brand_value if brand_value in valid_brands else None

def filter_concentration(concentration_value):
    valid_concentrations = ['솔리드 퍼퓸', '엑스트레 드 퍼퓸', '오 드 뚜왈렛', '오 드 코롱', '오 드 퍼퓸', '퍼퓸']
    if concentration_value is None:
        return None
    return concentration_value if concentration_value in valid_concentrations else None

def filter_day_night_score(day_night_value):
    valid_day_night = ["day", "night"]
    if day_night_value is None:
        return None
    if isinstance(day_night_value, str) and ',' in day_night_value:
        values = [v.strip() for v in day_night_value.split(',')]
        filtered_values = [v for v in values if v in valid_day_night]
        return ','.join(filtered_values) if filtered_values else None
    return day_night_value if day_night_value in valid_day_night else None

def filter_gender(gender_value):
    valid_genders = ['Female', 'Male', 'Unisex', 'unisex ']
    if gender_value is None:
        return None
    return gender_value if gender_value in valid_genders else None

def filter_season_score(season_value):
    valid_seasons = ['winter', 'spring', 'summer', 'fall']
    if season_value is None:
        return None
    return season_value if season_value in valid_seasons else None

def filter_sizes(sizes_value):
    valid_sizes = ['30', '50', '75', '100', '150']
    if sizes_value is None:
        return None
    if isinstance(sizes_value, str):
        numbers = re.findall(r'\d+', sizes_value)
        for num in numbers:
            if num in valid_sizes:
                return num
    return str(sizes_value) if str(sizes_value) in valid_sizes else None

def apply_meta_filters(parsed_json: dict) -> dict:
    """파싱된 JSON에 메타필터링 적용"""
    if not parsed_json or "error" in parsed_json:
        return parsed_json
    
    return {
        'brand': filter_brand(parsed_json.get('brand')),
        'concentration': filter_concentration(parsed_json.get('concentration')),
        'day_night_score': filter_day_night_score(parsed_json.get('day_night_score')),
        'gender': filter_gender(parsed_json.get('gender')),
        'season_score': filter_season_score(parsed_json.get('season_score')),
        'sizes': filter_sizes(parsed_json.get('sizes'))
    }

def build_pinecone_filter(filtered_json: dict) -> dict:
    """메타필터링 결과를 Pinecone filter dict로 변환"""
    pinecone_filter = {}
    if filtered_json.get("brand"):
        pinecone_filter["brand"] = {"$eq": filtered_json["brand"]}
    if filtered_json.get("sizes"):
        pinecone_filter["sizes"] = {"$eq": filtered_json["sizes"]}
    if filtered_json.get("season_score"):
        pinecone_filter["season_score"] = {"$eq": filtered_json["season_score"]}
    if filtered_json.get("gender"):
        pinecone_filter["gender"] = {"$eq": filtered_json["gender"]}
    if filtered_json.get("concentration"):
        pinecone_filter["concentration"] = {"$eq": filtered_json["concentration"]}
    if filtered_json.get("day_night_score"):
        pinecone_filter["day_night_score"] = {"$eq": filtered_json["day_night_score"]}
    return pinecone_filter

def query_pinecone(vector, filtered_json: dict, top_k: int = 5):
    """Pinecone 벡터 검색 + 메타데이터 필터 적용"""
    pinecone_filter = build_pinecone_filter(filtered_json)
    
    result = index.query(
        vector=vector,
        top_k=top_k,
        include_metadata=True,
        filter=pinecone_filter if pinecone_filter else None
    )
    return result

response_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 향수 전문가야. 사용자의 질문에 대해 검색된 향수 정보를 바탕으로 친절하고 전문적인 추천을 해줘.

추천할 때 다음을 포함해줘:
1. 왜 이 향수를 추천하는지
2. 향의 특징과 느낌
3. 어떤 상황에 적합한지
4. 가격대나 용량 관련 조언 (있다면)

자연스럽고 친근한 톤으로 답변해줘."""),
    ("user", """사용자 질문: {original_query}

검색된 향수 정보:
{search_results}

위 정보를 바탕으로 향수를 추천해줘.""")
])

def format_search_results(pinecone_results):
    """Pinecone 검색 결과를 텍스트로 포맷팅"""
    if not pinecone_results or not pinecone_results.get('matches'):
        return "검색된 향수가 없습니다."
    
    formatted_results = []
    for i, match in enumerate(pinecone_results['matches'], 1):
        metadata = match.get('metadata', {})
        score = match.get('score', 0)
        
        result_text = f"""
{i}. 향수명: {metadata.get('perfume_name', '정보없음')}
   - 브랜드: {metadata.get('brand', '정보없음')}
   - 성별: {metadata.get('gender', '정보없음')}
   - 용량: {metadata.get('sizes', '정보없음')}ml
   - 계절: {metadata.get('season_score', '정보없음')}
   - 사용시간: {metadata.get('day_night_score', '정보없음')}
   - 농도: {metadata.get('concentration', '정보없음')}
   - 유사도 점수: {score:.3f}
"""
        formatted_results.append(result_text.strip())
    
    return "\n\n".join(formatted_results)

def generate_response(original_query: str, search_results):
    """검색 결과를 바탕으로 최종 응답 생성"""
    try:
        formatted_results = format_search_results(search_results)
        
        chain = response_prompt | llm
        response = chain.invoke({
            "original_query": original_query,
            "search_results": formatted_results
        })
        
        return response.content
    except Exception as e:
        return f"응답 생성 중 오류가 발생했습니다: {str(e)}"

# ---------- 4) Agent Nodes ----------
def LLM_parser_node(state: AgentState) -> AgentState:
    """실제 RAG 파이프라인을 실행하는 LLM_parser 노드"""
    user_query = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            user_query = m.content
            break
    if not user_query:
        user_query = "(empty)"

    try:
        print(f"🔍 LLM_parser 실행: {user_query}")
        
        # 1단계: LLM으로 쿼리 파싱
        parsed_json = run_llm_parser(user_query)
        if "error" in parsed_json:
            error_msg = f"[LLM_parser] 쿼리 파싱 오류: {parsed_json['error']}"
            msgs = state["messages"] + [AIMessage(content=error_msg)]
            return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
        
        # 2단계: 메타필터 적용
        filtered_json = apply_meta_filters(parsed_json)
        
        # 3단계: 쿼리 벡터화
        query_vector = embeddings.embed_query(user_query)
        
        # 4단계: Pinecone 검색
        search_results = query_pinecone(query_vector, filtered_json, top_k=5)
        
        # 5단계: 최종 응답 생성
        final_response = generate_response(user_query, search_results)
        
        # 결과 요약
        summary = f"""[LLM_parser] RAG 파이프라인 완료 ✅

📊 파싱 결과: {json.dumps(parsed_json, ensure_ascii=False)}
🔍 필터링 결과: {json.dumps(filtered_json, ensure_ascii=False)}
🎯 검색된 향수 개수: {len(search_results.get('matches', []))}

💬 추천 결과:
{final_response}"""

        msgs = state["messages"] + [AIMessage(content=summary)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
        
    except Exception as e:
        error_msg = f"[LLM_parser] RAG 파이프라인 실행 중 오류: {str(e)}"
        msgs = state["messages"] + [AIMessage(content=error_msg)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}

def passthrough(name: str):
    def _node(state: AgentState) -> AgentState:
        payload = state.get("router_json") or {}
        summary = f"[{name}] handled. reason={payload.get('reason')} facets={payload.get('facets')} intent={payload.get('query_intent')}"
        msgs = state["messages"] + [AIMessage(content=summary)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
    return _node

FAQ_agent       = passthrough("FAQ_agent")
human_fallback  = passthrough("human_fallback")
price_agent     = passthrough("price_agent")
ML_agent        = passthrough("ML_agent")

# ---------- 5) Build Graph ----------
graph = StateGraph(AgentState)

graph.add_node("supervisor", supervisor_node)
graph.add_node("LLM_parser", LLM_parser_node)  # 실제 RAG 파이프라인 연결
graph.add_node("FAQ_agent", FAQ_agent)
graph.add_node("human_fallback", human_fallback)
graph.add_node("price_agent", price_agent)
graph.add_node("ML_agent", ML_agent)

graph.set_entry_point("supervisor")

# Conditional routing
def router_edge(state: AgentState) -> str:
    return state["next"] or "human_fallback"

graph.add_conditional_edges(
    "supervisor",
    router_edge,
    {
        "LLM_parser": "LLM_parser",
        "FAQ_agent": "FAQ_agent",
        "human_fallback": "human_fallback",
        "price_agent": "price_agent",
        "ML_agent": "ML_agent",
    },
)

# End states
for node in ["LLM_parser", "FAQ_agent", "human_fallback", "price_agent", "ML_agent"]:
    graph.add_edge(node, END)

app = graph.compile()

# ---------- 6) Batch Test ----------
TEST_QUERIES = [
    "입생로랑 여성용 50ml 겨울용 향수 추천해줘.",                 
    "디올 EDP로 가을 밤(야간)에 쓸 만한 향수 있어?",                
    "EDP랑 EDT 차이가 뭐야?",                                       
    "탑노트·미들노트·베이스노트가 각각 무슨 뜻이야?",               
    "오늘 점심 뭐 먹을까?",                                         
    "오늘 서울 날씨 어때?",                                         
    "샤넬 넘버5 50ml 최저가 알려줘.",                               
    "디올 소바쥬 가격 얼마야? 어디서 사는 게 제일 싸?",             
    "여름에 시원한 향수 추천해줘.",                                 
    "달달한 향 추천해줘.",                                         
]

def run_tests():
    for q in TEST_QUERIES:
        print("="*80)
        print("Query:", q)
        init: AgentState = {
            "messages": [HumanMessage(content=q)],
            "next": None,
            "router_json": None
        }
        out = app.invoke(init)
        ai_msgs = [m for m in out["messages"] if isinstance(m, AIMessage)]
        router_raw = ai_msgs[-2].content if len(ai_msgs) >= 2 else "(no router output)"
        agent_summary = ai_msgs[-1].content if ai_msgs else "(no agent output)"
        print("Router JSON:", router_raw)
        print("Agent summary:", agent_summary)

if __name__ == "__main__":
    run_tests()

Query: 입생로랑 여성용 50ml 겨울용 향수 추천해줘.
🔍 LLM_parser 실행: 입생로랑 여성용 50ml 겨울용 향수 추천해줘.
Router JSON: {
  "next": "LLM_parser",
  "reason": "The query contains multiple facets including brand, gender, size, and season.",
  "facet_count": 4,
  "facets": {
    "brand": "입생로랑",
    "season": "겨울",
    "gender": "여성",
    "sizes": "50ml",
    "day_night_score": null,
    "concentration": null
  },
  "scent_vibe": null,
  "query_intent": "other"
}
Agent summary: [LLM_parser] RAG 파이프라인 완료 ✅

📊 파싱 결과: {"brand": "입생로랑", "gender": "여성", "sizes": "50", "season_score": "겨울", "concentration": null, "day_night_score": null}
🔍 필터링 결과: {"brand": "입생로랑", "concentration": null, "day_night_score": null, "gender": null, "season_score": null, "sizes": "50"}
🎯 검색된 향수 개수: 5

💬 추천 결과:
안녕하세요! 겨울에 사용할 입생로랑 여성용 향수를 찾고 계시군요. 제가 추천드릴 향수는 **입생로랑의 오 드 뚜왈렛 50ml**입니다. 

### 추천 이유
이 향수는 겨울철에 특히 잘 어울리는 향으로, 따뜻하고 포근한 느낌을 주기 때문에 차가운 날씨에 잘 어울립니다. 

### 향의 특징과 느낌
입생로랑의 오 드 뚜왈렛은 상큼하면서도 부드러운 플로럴 노트가 특징입니다. 처음에는 신선한 과일 향이 느껴지다가, 시간이 지나면

In [ ]:
# ML 모델 버전
import json
import joblib
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from rank_bm25 import BM25Okapi
from typing import List, Dict, Tuple, Optional


class PerfumeRecommender:
    """향수 추천 시스템 클래스"""
    
    def __init__(self, 
                 model_pkl_path: str = "./models.pkl", 
                 perfume_json_path: str = "perfumes.json",
                 model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                 max_len: int = 256):
        
        self.model_name = model_name
        self.max_len = max_len
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"[Device] {self.device}")
        
        # 모델 및 데이터 로드
        self._load_ml_model(model_pkl_path)
        self._load_transformer_model()
        self._load_perfume_data(perfume_json_path)
        self._build_bm25_index()
    
    def _load_ml_model(self, pkl_path: str):
        """저장된 ML 모델 불러오기"""
        data = joblib.load(pkl_path)
        self.clf = data["classifier"]
        self.mlb = data["mlb"]
        self.thresholds = data["thresholds"]
        
        print(f"[Loaded model from {pkl_path}]")
        print(f"Labels: {list(self.mlb.classes_)}")
    
    def _load_transformer_model(self):
        """Transformer 모델 로드"""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.base_model = AutoModel.from_pretrained(self.model_name).to(self.device)
        self.base_model.eval()
    
    def _load_perfume_data(self, json_path: str):
        """향수 데이터 로드"""
        with open(json_path, "r", encoding="utf-8") as f:
            self.perfumes = json.load(f)
        print(f"[Loaded {len(self.perfumes)} perfumes from {json_path}]")
    
    def _build_bm25_index(self):
        """BM25 인덱스 구축"""
        self.corpus = [item.get("fragrances", "") for item in self.perfumes]
        tokenized_corpus = [doc.lower().split() for doc in self.corpus]
        self.bm25 = BM25Okapi(tokenized_corpus)
        print("[BM25 index built]")
    
    def encode_texts(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """텍스트를 임베딩으로 변환"""
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            enc = self.tokenizer(
                batch, 
                padding=True, 
                truncation=True, 
                max_length=self.max_len, 
                return_tensors="pt"
            ).to(self.device)
            
            with torch.no_grad():
                model_out = self.base_model(**enc)
                emb = model_out.last_hidden_state.mean(dim=1)
            
            all_embeddings.append(emb.cpu().numpy())
        
        return np.vstack(all_embeddings)
    
    def predict_labels(self, text: str, topk: int = 3, use_thresholds: bool = True) -> List[str]:
        """텍스트에서 향수 라벨 예측"""
        emb = self.encode_texts([text], batch_size=1)
        proba = self.clf.predict_proba(emb)[0]
        
        if use_thresholds and self.thresholds:
            # threshold 기반 선택
            pick = [
                i for i, p in enumerate(proba) 
                if p >= self.thresholds.get(self.mlb.classes_[i], 0.5)
            ]
            # threshold를 넘는 것이 없으면 topk 선택
            if not pick:
                pick = np.argsort(-proba)[:topk]
        else:
            # 상위 topk 선택
            pick = np.argsort(-proba)[:topk]
        
        return [self.mlb.classes_[i] for i in pick]
    
    def search_perfumes(self, labels: List[str], top_n: int = 5) -> List[Tuple[int, float, Dict]]:
        """BM25를 사용해 향수 검색"""
        query = " ".join(labels)
        tokenized_query = query.lower().split()
        scores = self.bm25.get_scores(tokenized_query)
        
        # 상위 N개 인덱스 선택
        top_idx = np.argsort(scores)[-top_n:][::-1]
        
        results = []
        for idx in top_idx:
            results.append((idx, scores[idx], self.perfumes[idx]))
        
        return results
    
    def recommend(self, 
                  user_text: str, 
                  topk_labels: int = 4, 
                  top_n_perfumes: int = 5,
                  use_thresholds: bool = True,
                  verbose: bool = True) -> Dict:
        """전체 추천 파이프라인"""
        
        # 1. ML 모델로 라벨 예측
        predicted_labels = self.predict_labels(
            user_text, 
            topk=topk_labels, 
            use_thresholds=use_thresholds
        )
        
        # 2. BM25로 향수 검색
        search_results = self.search_perfumes(predicted_labels, top_n=top_n_perfumes)
        
        if verbose:
            print("=== ML 예측 라벨 ===")
            print(predicted_labels)
            print(f"\n=== BM25 Top-{top_n_perfumes} 결과 ===")
            
            for rank, (idx, score, perfume) in enumerate(search_results, 1):
                print(f"[Rank {rank}] Score: {score:.2f}")
                print(f"  Brand      : {perfume.get('brand', 'N/A')}")
                print(f"  Name       : {perfume.get('name_perfume', 'N/A')}")
                print(f"  Fragrances : {perfume.get('fragrances', 'N/A')}")
                print()
        
        return {
            "user_input": user_text,
            "predicted_labels": predicted_labels,
            "recommendations": [
                {
                    "rank": rank,
                    "score": score,
                    "brand": perfume.get('brand', 'N/A'),
                    "name": perfume.get('name_perfume', 'N/A'),
                    "fragrances": perfume.get('fragrances', 'N/A'),
                    "perfume_data": perfume
                }
                for rank, (idx, score, perfume) in enumerate(search_results, 1)
            ]
        }


# 사용 예시
def main():
    # 추천 시스템 초기화
    recommender = PerfumeRecommender()
    
    # 사용자 입력 예시들
    test_inputs = [
        "시트러스하고 프루티한 향수 추천해줘",
        "로맨틱하고 플로랄한 향 원해",
        "우디하고 스파이시한 향수",
        "깔끔하고 상쾌한 향"
    ]
    
    for user_input in test_inputs:
        print(f"\n{'='*50}")
        print(f"사용자 입력: {user_input}")
        print(f"{'='*50}")
        
        # 추천 실행
        result = recommender.recommend(
            user_text=user_input,
            topk_labels=4,
            top_n_perfumes=3,
            verbose=True
        )


if __name__ == "__main__":
    main()
이걸 사용해서 나머지 agent들을 통합해줘

SyntaxError: invalid syntax (3917280898.py, line 52)

In [13]:
# pip install -U langchain langgraph langchain-openai tiktoken python-dotenv pinecone-client
import os, json, re
from typing import TypedDict, List, Optional, Dict, Any
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from pinecone import Pinecone

# 환경 변수 로드
load_dotenv()

# ---------- 0) Config ----------
os.environ.setdefault("OPENAI_API_KEY", "PUT_YOUR_KEY_HERE")  # or set in env
MODEL_NAME = "gpt-4o-mini"  # keep it small & fast for routing

SUPERVISOR_SYSTEM_PROMPT = """
You are the "Perfume Recommendation Supervisor (Router)". Analyze the user's query (Korean or English) and route to exactly ONE agent below.

[Agents]
- LLM_parser         : Parses/normalizes multi-facet queries (2+ product facets).
- FAQ_agent          : Perfume knowledge / definitions / differences / general questions.
- human_fallback     : Non-perfume or off-topic queries.
- price_agent        : Price-only intents (cheapest, price, buy, discount, etc.).
- ML_agent           : Single-preference recommendations (mood/season vibe like "fresh summer", "sweet", etc.).

[Facets to detect ("product facets")]
- brand            (e.g., Chanel, Dior, Creed)
- season           (spring/summer/fall/winter; "for summer/winter")
- gender           (male/female/unisex)
- sizes            (volume in ml: 30/50/100 ml)
- day_night_score  (day/night/daily/office/club, etc.)
- concentration    (EDT/EDP/Extrait/Parfum/Cologne)

[Price intent keywords (not exhaustive)]
- Korean: 가격, 최저가, 얼마, 가격대, 구매, 판매, 할인, 어디서 사, 배송비
- English: price, cost, cheapest, buy, purchase, discount

[FAQ examples]
- Differences between EDP vs EDT, note definitions, longevity/projection, brand/line info.

[Single-preference (ML_agent) examples]
- "Recommend a cool perfume for summer", "Recommend a sweet scent", "One citrusy fresh pick"
  (= 0–1 of the above facets mentioned; primarily taste/mood/situation).

[Routing rules (priority)]
1) Non-perfume / off-topic → human_fallback
2) Clear price-only intent (even if one facet is present as context) → price_agent
   e.g., "Chanel No. 5 50ml cheapest price?" → price_agent
3) Count product facets in the query:
   - If facets ≥ 2 → LLM_parser
4) Otherwise (single-topic queries):
   - Perfume knowledge/definitions → FAQ_agent
   - Single taste/mood recommendation → ML_agent
5) Tie-breakers:
   - If price intent is clear → price_agent
   - If facets ≥ 2 → LLM_parser
   - Else: knowledge → FAQ_agent, taste → ML_agent

[Output format]
Return ONLY this JSON (no extra text):
{{
  "next": "<LLM_parser|FAQ_agent|human_fallback|price_agent|ML_agent>",
  "reason": "<one short English sentence>",
  "facet_count": <integer>,
  "facets": {{
    "brand": "<value or null>",
    "season": "<value or null>",
    "gender": "<value or null>",
    "sizes": "<value or null>",
    "day_night_score": "<value or null>",
    "concentration": "<value or null>"
  }},
  "scent_vibe": "<value if detected, else null>",
  "query_intent": "<price|faq|scent_pref|non_perfume|other>"
}}
""".strip()

# ---------- 1) State ----------
class AgentState(TypedDict):
    messages: List[BaseMessage]           # conversation log
    next: Optional[str]                   # routing decision key
    router_json: Optional[Dict[str, Any]] # parsed JSON from router

# ---------- 2) LLM 초기화 ----------
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Pinecone 초기화
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("perfume-vectordb2")

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SUPERVISOR_SYSTEM_PROMPT),
        ("user", "{query}")
    ]
)

def supervisor_node(state: AgentState) -> AgentState:
    """Call the router LLM and return parsed JSON + routing target."""
    user_query = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            user_query = m.content
            break
    if not user_query:
        user_query = "(empty)"

    chain = router_prompt | llm
    ai = chain.invoke({"query": user_query})
    text = ai.content

    # JSON strict parse
    chosen = "human_fallback"
    parsed: Dict[str, Any] = {}
    try:
        parsed = json.loads(text)
        maybe = parsed.get("next")
        if isinstance(maybe, str) and maybe in {"LLM_parser","FAQ_agent","human_fallback","price_agent","ML_agent"}:
            chosen = maybe
    except Exception:
        parsed = {"error": "invalid_json", "raw": text}

    msgs = state["messages"] + [AIMessage(content=text)]
    return {
        "messages": msgs,
        "next": chosen,
        "router_json": parsed
    }

# ---------- 3) RAG Pipeline Functions ----------
parse_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 향수 쿼리 파서야.
사용자의 질문에서 다음 정보를 JSON 형식으로 추출해줘:
- brand: 브랜드명 (예: 샤넬, 디올, 입생로랑 등)
- concentration: (퍼퓸, 코롱 등)
- day_night_score: 사용시간 (주간, 야간, 데일리 등)
- gender: 성별 (남성, 여성, 유니섹스)
- season_score: 계절 (봄, 여름, 가을, 겨울)
- sizes: 용량 (30ml, 50ml, 100ml 등) 단위는 무시하고 숫자만

없는 값은 null로 두고, 반드시 유효한 JSON 형식으로만 응답해줘.

예시:
{{"brand": "샤넬", "gender": null, "sizes": "50", "season_score": null, "concentration": null, "day_night_score": null}}"""),
    ("user", "{query}")
])

def run_llm_parser(query: str):
    """사용자 쿼리를 JSON으로 파싱"""
    try:
        chain = parse_prompt | llm
        ai_response = chain.invoke({"query": query})
        response_text = ai_response.content.strip()

        # JSON 부분만 추출
        if "```json" in response_text:
            response_text = response_text.split("```json")[1].split("```")[0].strip()
        elif "```" in response_text:
            response_text = response_text.split("```")[1].strip()

        parsed = json.loads(response_text)
        return parsed
    except Exception as e:
        return {"error": f"파싱 오류: {str(e)}"}

# 메타필터 함수들
def filter_brand(brand_value):
    valid_brands = [
        '겔랑', '구찌', '끌로에', '나르시소 로드리게즈', '니샤네', '도르세', '디올', '딥티크', '랑콤',
        '로라 메르시에', '로에베', '록시땅', '르 라보', '메모', '메종 마르지엘라', '메종 프란시스 커정',
        '멜린앤게츠', '미우미우', '바이레도', '반클리프 아펠', '버버리', '베르사체', '불가리', '비디케이',
        '산타 마리아 노벨라', '샤넬', '세르주 루텐', '시슬리 코스메틱', '아쿠아 디 파르마', '에따 리브르 도량쥬',
        '에르메스', '에스티 로더', '엑스 니힐로', '이니시오 퍼퓸', '이솝', '입생로랑', '제르조프', '조 말론',
        '조르지오 아르마니', '줄리엣 헤즈 어 건', '지방시', '질 스튜어트', '크리드', '킬리안', '톰 포드',
        '티파니앤코', '퍼퓸 드 말리', '펜할리곤스', '프라다', '프레데릭 말'
    ]
    if brand_value is None:
        return None
    return brand_value if brand_value in valid_brands else None

def filter_concentration(concentration_value):
    valid_concentrations = ['솔리드 퍼퓸', '엑스트레 드 퍼퓸', '오 드 뚜왈렛', '오 드 코롱', '오 드 퍼퓸', '퍼퓸']
    if concentration_value is None:
        return None
    return concentration_value if concentration_value in valid_concentrations else None

def filter_day_night_score(day_night_value):
    valid_day_night = ["day", "night"]
    if day_night_value is None:
        return None
    if isinstance(day_night_value, str) and ',' in day_night_value:
        values = [v.strip() for v in day_night_value.split(',')]
        filtered_values = [v for v in values if v in valid_day_night]
        return ','.join(filtered_values) if filtered_values else None
    return day_night_value if day_night_value in valid_day_night else None

def filter_gender(gender_value):
    valid_genders = ['Female', 'Male', 'Unisex', 'unisex ']
    if gender_value is None:
        return None
    return gender_value if gender_value in valid_genders else None

def filter_season_score(season_value):
    valid_seasons = ['winter', 'spring', 'summer', 'fall']
    if season_value is None:
        return None
    return season_value if season_value in valid_seasons else None

def filter_sizes(sizes_value):
    valid_sizes = ['30', '50', '75', '100', '150']
    if sizes_value is None:
        return None
    if isinstance(sizes_value, str):
        numbers = re.findall(r'\d+', sizes_value)
        for num in numbers:
            if num in valid_sizes:
                return num
    return str(sizes_value) if str(sizes_value) in valid_sizes else None

def apply_meta_filters(parsed_json: dict) -> dict:
    """파싱된 JSON에 메타필터링 적용"""
    if not parsed_json or "error" in parsed_json:
        return parsed_json
    
    return {
        'brand': filter_brand(parsed_json.get('brand')),
        'concentration': filter_concentration(parsed_json.get('concentration')),
        'day_night_score': filter_day_night_score(parsed_json.get('day_night_score')),
        'gender': filter_gender(parsed_json.get('gender')),
        'season_score': filter_season_score(parsed_json.get('season_score')),
        'sizes': filter_sizes(parsed_json.get('sizes'))
    }

def build_pinecone_filter(filtered_json: dict) -> dict:
    """메타필터링 결과를 Pinecone filter dict로 변환"""
    pinecone_filter = {}
    if filtered_json.get("brand"):
        pinecone_filter["brand"] = {"$eq": filtered_json["brand"]}
    if filtered_json.get("sizes"):
        pinecone_filter["sizes"] = {"$eq": filtered_json["sizes"]}
    if filtered_json.get("season_score"):
        pinecone_filter["season_score"] = {"$eq": filtered_json["season_score"]}
    if filtered_json.get("gender"):
        pinecone_filter["gender"] = {"$eq": filtered_json["gender"]}
    if filtered_json.get("concentration"):
        pinecone_filter["concentration"] = {"$eq": filtered_json["concentration"]}
    if filtered_json.get("day_night_score"):
        pinecone_filter["day_night_score"] = {"$eq": filtered_json["day_night_score"]}
    return pinecone_filter
@tool
def price_tool(user_query: str) -> str: #이거 price_agent 함수야
    """A tool that uses the Naver Shopping API to look up perfume prices (results are returned as formatted strings)"""
    
    url = "https://openapi.naver.com/v1/search/shop.json"
    headers = {
        "X-Naver-Client-Id": naver_client_id,
        "X-Naver-Client-Secret": naver_client_secret
    }
    params = {"query": user_query, "display": 5, "sort": "sim"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
    except Exception as e:
        return f"❌ 요청 오류: {e}"
    
    if response.status_code != 200:
        return f"❌ API 오류: {response.status_code}"
    
    data = response.json()
    if not data or "items" not in data or len(data["items"]) == 0:
        return f"😔 '{user_query}'에 대한 검색 결과가 없습니다."
    
    # HTML 태그 제거 함수
    def remove_html_tags(text: str) -> str:
        return re.sub(r"<[^>]+>", "", text)
    
    # 상위 3개만 정리
    products = data["items"][:3]
    output = f"🔍 '{user_query}' 검색 결과:\n\n"
    for i, item in enumerate(products, 1):
        title = remove_html_tags(item.get("title", ""))
        lprice = item.get("lprice", "0")
        mall = item.get("mallName", "정보 없음")
        link = item.get("link", "정보 없음")
        
        output += f"📦 {i}. {title}\n"
        if lprice != "0":
            output += f"   💰 가격: {int(lprice):,}원\n"
        output += f"   🏪 판매처: {mall}\n"
        output += f"   🔗 링크: {link}\n\n"
    
    return output

def human_fallback(state: dict) -> str:
    """향수 관련 복잡한 질문에 대한 기본 응답"""
    query = state.get("input", "")
    return (
        f"❓ '{query}' 더 명확한 설명이 필요합니다.\n"
        f"👉 질문을 구체적으로 다시 작성해 주세요.\n"
        f"💡 또는 향수에 관한 멋진 질문을 해보시는 건 어떨까요?")

def query_pinecone(vector, filtered_json: dict, top_k: int = 5):
    """Pinecone 벡터 검색 + 메타데이터 필터 적용"""
    pinecone_filter = build_pinecone_filter(filtered_json)
    
    result = index.query(
        vector=vector,
        top_k=top_k,
        include_metadata=True,
        filter=pinecone_filter if pinecone_filter else None
    )
    return result

response_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 향수 전문가야. 사용자의 질문에 대해 검색된 향수 정보를 바탕으로 친절하고 전문적인 추천을 해줘.

추천할 때 다음을 포함해줘:
1. 왜 이 향수를 추천하는지
2. 향의 특징과 느낌
3. 어떤 상황에 적합한지
4. 가격대나 용량 관련 조언 (있다면)

자연스럽고 친근한 톤으로 답변해줘."""),
    ("user", """사용자 질문: {original_query}

검색된 향수 정보:
{search_results}

위 정보를 바탕으로 향수를 추천해줘.""")
])

def format_search_results(pinecone_results):
    """Pinecone 검색 결과를 텍스트로 포맷팅"""
    if not pinecone_results or not pinecone_results.get('matches'):
        return "검색된 향수가 없습니다."
    
    formatted_results = []
    for i, match in enumerate(pinecone_results['matches'], 1):
        metadata = match.get('metadata', {})
        score = match.get('score', 0)
        
        result_text = f"""
{i}. 향수명: {metadata.get('perfume_name', '정보없음')}
   - 브랜드: {metadata.get('brand', '정보없음')}
   - 성별: {metadata.get('gender', '정보없음')}
   - 용량: {metadata.get('sizes', '정보없음')}ml
   - 계절: {metadata.get('season_score', '정보없음')}
   - 사용시간: {metadata.get('day_night_score', '정보없음')}
   - 농도: {metadata.get('concentration', '정보없음')}
   - 유사도 점수: {score:.3f}
"""
        formatted_results.append(result_text.strip())
    
    return "\n\n".join(formatted_results)

def generate_response(original_query: str, search_results):
    """검색 결과를 바탕으로 최종 응답 생성"""
    try:
        formatted_results = format_search_results(search_results)
        
        chain = response_prompt | llm
        response = chain.invoke({
            "original_query": original_query,
            "search_results": formatted_results
        })
        
        return response.content
    except Exception as e:
        return f"응답 생성 중 오류가 발생했습니다: {str(e)}"

# ---------- 4) Agent Nodes ----------
def LLM_parser_node(state: AgentState) -> AgentState:
    """실제 RAG 파이프라인을 실행하는 LLM_parser 노드"""
    user_query = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            user_query = m.content
            break
    if not user_query:
        user_query = "(empty)"

    try:
        print(f"🔍 LLM_parser 실행: {user_query}")
        
        # 1단계: LLM으로 쿼리 파싱
        parsed_json = run_llm_parser(user_query)
        if "error" in parsed_json:
            error_msg = f"[LLM_parser] 쿼리 파싱 오류: {parsed_json['error']}"
            msgs = state["messages"] + [AIMessage(content=error_msg)]
            return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
        
        # 2단계: 메타필터 적용
        filtered_json = apply_meta_filters(parsed_json)
        
        # 3단계: 쿼리 벡터화
        query_vector = embeddings.embed_query(user_query)
        
        # 4단계: Pinecone 검색
        search_results = query_pinecone(query_vector, filtered_json, top_k=5)
        
        # 5단계: 최종 응답 생성
        final_response = generate_response(user_query, search_results)
        
        # 결과 요약
        summary = f"""[LLM_parser] RAG 파이프라인 완료 ✅

📊 파싱 결과: {json.dumps(parsed_json, ensure_ascii=False)}
🔍 필터링 결과: {json.dumps(filtered_json, ensure_ascii=False)}
🎯 검색된 향수 개수: {len(search_results.get('matches', []))}

💬 추천 결과:
{final_response}"""

        msgs = state["messages"] + [AIMessage(content=summary)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
        
    except Exception as e:
        error_msg = f"[LLM_parser] RAG 파이프라인 실행 중 오류: {str(e)}"
        msgs = state["messages"] + [AIMessage(content=error_msg)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}

def passthrough(name: str):
    def _node(state: AgentState) -> AgentState:
        payload = state.get("router_json") or {}
        summary = f"[{name}] handled. reason={payload.get('reason')} facets={payload.get('facets')} intent={payload.get('query_intent')}"
        msgs = state["messages"] + [AIMessage(content=summary)]
        return {"messages": msgs, "next": None, "router_json": state.get("router_json")}
    return _node

price_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a perfume price specialist assistant.
    
When users ask about perfume prices:
1. Use the price_tool to search for current prices
2. Always respond in Korean
3. Format results nicely with emojis and clear information
4. Be helpful and friendly
    
If you can't find price information, politely explain and suggest alternative searches."""),
    ("placeholder", "{messages}"),
])

price_agent = create_react_agent(
    llm, 
    [price_tool],
    prompt=price_prompt
)




# --- 7) 에이전트 호출 래퍼 ---
def price_agent_node(state: AgentState) -> dict:
    """Price agent 호출"""
    result = price_agent.invoke(state)
    return {"messages": result["messages"]}

def FAQ_agent_node(state: AgentState) -> dict:
    """FAQ_agent_nodet 호출"""
    result = FAQ_agent.invoke(state)
    return {"messages": result["messages"]}

def ML_agent_node(state: AgentState) -> dict:
    """ML_agent 호출"""
    result = ML_agent.invoke(state)
    return {"messages": result["messages"]}

# FAQ_agent       = passthrough("FAQ_agent")
# human_fallback  = passthrough("human_fallback")
# price_agent     = passthrough("price_agent")
# ML_agent        = passthrough("ML_agent")

# ---------- 5) Build Graph ----------
graph = StateGraph(AgentState)

graph.add_node("supervisor", supervisor_node)
graph.add_node("LLM_parser", LLM_parser_node)  # 실제 RAG 파이프라인 연결
graph.add_node("FAQ_agent", FAQ_agent_node)
graph.add_node("human_fallback", human_fallback)
graph.add_node("price_agent", price_agent_node)
graph.add_node("ML_agent", ML_agent_node)

graph.set_entry_point("supervisor")

# Conditional routing
def router_edge(state: AgentState) -> str:
    return state["next"] or "human_fallback"

graph.add_conditional_edges(
    "supervisor",
    router_edge,
    {
        "LLM_parser": "LLM_parser",
        "FAQ_agent": "FAQ_agent",
        "human_fallback": "human_fallback",
        "price_agent": "price_agent",
        "ML_agent": "ML_agent",
    },
)

# End states
for node in ["LLM_parser", "FAQ_agent", "human_fallback", "price_agent", "ML_agent"]:
    graph.add_edge(node, END)

app = graph.compile()

# ---------- 6) Batch Test ----------
TEST_QUERIES = [
    "입생로랑 여성용 50ml 겨울용 향수 추천해줘.",                 
    "디올 EDP로 가을 밤(야간)에 쓸 만한 향수 있어?",                
    "EDP랑 EDT 차이가 뭐야?",                                       
    "탑노트·미들노트·베이스노트가 각각 무슨 뜻이야?",               
    "오늘 점심 뭐 먹을까?",                                         
    "오늘 서울 날씨 어때?",                                         
    "샤넬 넘버5 50ml 최저가 알려줘.",                               
    "디올 소바쥬 가격 얼마야? 어디서 사는 게 제일 싸?",             
    "여름에 시원한 향수 추천해줘.",                                 
    "달달한 향 추천해줘.",                                         
]

def run_tests():
    for q in TEST_QUERIES:
        print("="*80)
        print("Query:", q)
        init: AgentState = {
            "messages": [HumanMessage(content=q)],
            "next": None,
            "router_json": None
        }
        out = app.invoke(init)
        ai_msgs = [m for m in out["messages"] if isinstance(m, AIMessage)]
        router_raw = ai_msgs[-2].content if len(ai_msgs) >= 2 else "(no router output)"
        agent_summary = ai_msgs[-1].content if ai_msgs else "(no agent output)"
        print("Router JSON:", router_raw)
        print("Agent summary:", agent_summary)

if __name__ == "__main__":
    # 환경 변수 확인
    print("🔧 환경 변수 확인:")
    print(f"OPENAI_API_KEY: {'✅ 설정됨' if os.getenv('OPENAI_API_KEY') else '❌ 미설정'}")
    print(f"PINECONE_API_KEY: {'✅ 설정됨' if os.getenv('PINECONE_API_KEY') else '❌ 미설정'}")
    print(f"NAVER_CLIENT_ID: {'✅ 설정됨' if os.getenv('NAVER_CLIENT_ID') else '❌ 미설정'}")
    print(f"NAVER_CLIENT_SECRET: {'✅ 설정됨' if os.getenv('NAVER_CLIENT_SECRET') else '❌ 미설정'}")
    print()
    
    print()
    
    run_tests()

🔧 환경 변수 확인:
OPENAI_API_KEY: ✅ 설정됨
PINECONE_API_KEY: ✅ 설정됨
NAVER_CLIENT_ID: ✅ 설정됨
NAVER_CLIENT_SECRET: ✅ 설정됨


Query: 입생로랑 여성용 50ml 겨울용 향수 추천해줘.
🔍 LLM_parser 실행: 입생로랑 여성용 50ml 겨울용 향수 추천해줘.
Router JSON: {
  "next": "LLM_parser",
  "reason": "The query contains multiple facets including brand, gender, size, and season.",
  "facet_count": 4,
  "facets": {
    "brand": "입생로랑",
    "season": "겨울",
    "gender": "여성",
    "sizes": "50ml",
    "day_night_score": null,
    "concentration": null
  },
  "scent_vibe": null,
  "query_intent": "other"
}
Agent summary: [LLM_parser] RAG 파이프라인 완료 ✅

📊 파싱 결과: {"brand": "입생로랑", "gender": "여성", "sizes": "50", "season_score": "겨울", "concentration": null, "day_night_score": null}
🔍 필터링 결과: {"brand": "입생로랑", "concentration": null, "day_night_score": null, "gender": null, "season_score": null, "sizes": "50"}
🎯 검색된 향수 개수: 5

💬 추천 결과:
안녕하세요! 입생로랑의 겨울용 여성 향수를 찾고 계시군요. 제가 추천드릴 향수는 입생로랑의 **오 드 뚜왈렛** 50ml입니다. 

### 추천 이유
이 향수는 겨울철에 특히 잘 어울리는 향으로, 따뜻하고 포근한 느낌을 주기 때문

AttributeError: 'function' object has no attribute 'invoke'